# Merging Annotations


##  1. PLoS Methods

In [2]:
import os
from os import listdir
from os.path import isfile, join


# we are interested only on annotations set of:  (software_usage and purpose annotations)

interest_list = ['Application_Usage','ProgrammingEnvironment_Usage',
                 'PlugIn_Usage','OperatingSystem_Usage', 'Purpose_Analysis',
                 'Purpose_Modelling', 'Purpose_Stimulation', 
                 'Purpose_DataCollection', 'Purpose_DataPreProcss', 
                 'Purpose_Simulation', 'Purpose_Visualization', 'Purpose_Programming', 
                 'Application_Creation', 'PlugIn_Creation',
                 'Application_Deposition', 'PlugIn_Deposition', 'SoftwareCoreference_Deposition',
                 'Application_Mention','ProgrammingEnvironment_Mention','OperatingSystem_Mention','PlugIn_Mention',
                 'Version','Developer', 'URL', 'License','Citation', 'Abbreviation', 'AlternativeName', 'Release', 'Extension']

# file path
path = 'SoMeSci/PLoS_methods/'

# gather list of .ann PLoS_methods files ---------------------------------------------
PLOS_filesList = []

# iterating over all files in the dir
for file_name in os.listdir(path):
    
    # if the file is .ann
    if file_name.endswith('.ann'):
        PLOS_filesList.append(file_name)
    else:
        continue

PLOS_filesList.sort() 

print('.ann PLosMethods file count : ', len(PLOS_filesList), '\n')
print('Firts 5 files: ',PLOS_filesList[:5])

# gather list of .ann PLoS_methods files ---------------------------------------------

.ann PLosMethods file count :  480 

Firts 5 files:  ['PMC1088287.ann', 'PMC1657052.ann', 'PMC1831734.ann', 'PMC1880851.ann', 'PMC1939732.ann']


In [3]:
#interest_list

In [4]:
"""

There are different kinds of annotations in the .ann file. 
-----------------------------------------------------------

TYPES OF ANNOTATIONS

T - text bound annotations
R - relations
E - event
A - attribute
M - modification
N - Normalization 
-----------------------------------------------------------

We are interested in "text bound annotations only" that indicate software_usage and software_purpose.

Therefore get a list of all annotations in the .ann file iff:
        
        1. annotations are text bound ( start with T) & 
        2. annotations indicate software_usage or software_purpose
        
the result is then stored in a dictionary 

where,

    key of dict == name of the file 
    Value of dict == list of list of annotation lines in the file
"""

# stores ORIGIAL list of annotations in a file (BEFORE MERGING)
fileAn_dict = {}

# store all annotations in the file ( format: list-of-list)
annotation_filelist = []

# loop over each file 
for indx, file in enumerate(PLOS_filesList[:]):
    
    # path to each file
    file_path = path + file
    
    with open(file_path, "r") as a_file:
        
        # stores annotation line 
        annotation_line = []
        
        
        # check if the annotation is what we want (usage, purpose, starts with T )
        for line in a_file:
            
            # grab the type of annotation
            annotataion = line.split()[1:2][0]
            
            
            if (line.startswith('T') & (annotataion in interest_list)):

                if ( line.split()[2:4] ) not in annotation_line: 
                    
                    #print(line.split()[2:4])
                    
                    annotation_line.append(line.split())
                    #print(indx, file, line.split())

                else:
                    pass
        
        fileAn_dict[file] = annotation_line     

In [5]:
#fileAn_dict

In [6]:
print('The size of the mapping dictionary: ', len(fileAn_dict))
print('ALl files included!')

The size of the mapping dictionary:  480
ALl files included!


## Merging Annotations

In [7]:
'''

each annotation line has a form of:

     [Tx | Annotation | START | STOP | nameOfSoftware]
     
     example: 'T2', 'Application_Usage', '13536', '13539', 'SAS'

mergeList() merges two lines of annotations if their start is the same

'''

def mergeList(list_1, list_2):
    
    #stores merged annotations
    result_list = []
    
    '''
    
     list_1[2:3] is START number 
     merge two annotations if their start number is the same
    
    '''
    
    for x, y in [(x,y) for x in list_1[2:3] for y in list_2[2:3]]:
        
        
        
        # if the starting positions are the same x=y
        if( x == y):
            
            #print(x)
            
            #get id of the firt annotation 
            result_list.append(list_1[0])
            
            #merge annotations by :
            result_list.append(list_1[1]+':'+list_2[1])
            
            # start number
            result_list.append(list_1[2])
            
            #end number
            result_list.append(list_1[3])
            
            # get name of the software
            software_name_splitted = list_1[4:]
            software_name_joined = ' '.join(software_name_splitted)
            
            result_list.append(software_name_joined)
    
            
        else:
            pass
    return result_list

In [8]:
# stores MERGED LIST of annotations in each file
merged_dict = {}

#stores list of list of all annotations
all_annotations_list = []


# for each file in the dict
for key in list(fileAn_dict)[:]:
    
    # stores list of merged annotations
    merged_annotationlist_perfile = []
    
    #print(key)
    """
    compare all annotations to each other and grab those that share the same "start number"
    
    """
    
    lookup_list1 = []
    for ls1 in fileAn_dict.get(key):
            
            
            lookup_list2 = []
            for ls2 in fileAn_dict.get(key):
                
                
                # compare each annotation with another but not to itself
                if (ls1 != ls2) :
                    
                    
                    # merge annotations
                    r = mergeList(ls1,ls2)
                    
                    # if the 
                    if len(r) !=0 :
                        
                        #print(ls1[0],ls2[0], ls1[2:4], ls2[2:4])
                        
         
                        if (r[2] not in lookup_list1 ) | (r[3] not in lookup_list1 ):
                            
                            merged_annotationlist_perfile.append(r) 
                            all_annotations_list.append(r)
                            
                            lookup_list2.extend(r[2:4])
                            
                        else:
                            pass
            # save already merged list on the lookup list            
            lookup_list1.extend(lookup_list2)  
            #break
                        
                        
    merged_dict[key] = merged_annotationlist_perfile

In [9]:
#for file in list(merged_dict)[:]:
#    print('file name:', file, '|  ',merged_dict.get(file), '\n\n')

## PLoS:  writing merged annotations back to *.ann files

In [10]:
# fileAn_dict -> contains file name -vs- list of  (original) annotations 


print("# of files with ORIGINAL annotations:", len(fileAn_dict))

print("# of files with MERGED annotations:", len(merged_dict))

# merged_dict -> contains file name -vs- list of  (merged) annotations 

# of files with ORIGINAL annotations: 480
# of files with MERGED annotations: 480


In [11]:
for file in list(merged_dict):
    
    path2 = 'SoMeSci/PLoS_methodsClean/'
    file_path = path2 + file
    
    #print(file_path)
    
    with open(file_path, "w") as f1:
        
        
        for line in merged_dict.get(file):
            
            txt = '\t'.join(line)
            
            #print('Writing To ->',file, '<-',txt)
            
            #write merged annotation to the file 
            f1.write(txt+ '\n')
            
        #print("-------------------------------------------------")

### Store the merged annotation lists Into a Data Frame

In [12]:
import pandas as pd

df_PLOS = pd.DataFrame(columns=['code', 'annotation','start','stop','software'], data=all_annotations_list)
#df_PLOS

### Save the df_PLOS image

In [13]:
# get head and tail of the df
PLOS_head = df_PLOS.head(10)
PLOS_tail = df_PLOS.tail(10)


import dataframe_image as dfi


PLOS_head = PLOS_head.style.background_gradient()
dfi.export(PLOS_head,"PLOS_head_.png")

PLOS_tail = PLOS_tail.style.background_gradient()
dfi.export(PLOS_tail,"PLOS_tail_.png")

df_PLOS.to_csv(r'PLOS_.csv', index = False)

[0110/102641.622912:ERROR:sandbox_linux.cc(376)] InitializeSandbox() called with multiple threads in process gpu-process.
[0110/102641.714915:INFO:headless_shell.cc(653)] Written to file /tmp/tmpg9w0ynx8/temp.png.
[0110/102641.849804:ERROR:sandbox_linux.cc(376)] InitializeSandbox() called with multiple threads in process gpu-process.
[0110/102641.973368:INFO:headless_shell.cc(653)] Written to file /tmp/tmp7d8dvn0l/temp.png.


## 2. PubMed Full Text

In [14]:
import os
from os import listdir
from os.path import isfile, join


interest_list = ['Application_Usage','ProgrammingEnvironment_Usage','PlugIn_Usage','OperatingSystem_Usage', 'Purpose_Analysis','Purpose_Modelling', 'Purpose_Stimulation', 
                 'Purpose_DataCollection', 'Purpose_DataPreProcss', 'Purpose_Simulation', 'Purpose_Visualization', 
                 'Purpose_Programming', 'Application_Creation', 'PlugIn_Creation',
                 'Application_Deposition', 'PlugIn_Deposition', 'SoftwareCoreference_Deposition',
                 'Application_Mention','ProgrammingEnvironment_Mention','OperatingSystem_Mention','PlugIn_Mention',
                 'Version','Developer', 'URL', 'License','Citation', 'Abbreviation', 'AlternativeName', 'Release', 'Extension']


#  1. get a list of PubMed file names---------------------------------------------------------------------

mypath = "SoMeSci/Pubmed_fulltext/"

Pubmed_filesList = []
  
# iterating over all files in the dir
for file_name in os.listdir(mypath):
    
    # if the file is .ann
    if file_name.endswith('.ann'):
        Pubmed_filesList.append(file_name)
    else:
        continue

Pubmed_filesList.sort() 

#print('First 5 files: ',Pubmed_filesList[:5])


# 2. get file name and list of annotations into a dict-------------------------------------------------------


# stores ORIGIAL list of annotations in a file (BEFORE MERGING)
fileAn_dict2 = {}

# store all annotations in the file ( format: list-of-list)
annotation_filelist2 = []

# loop over each file 
for indx, file in enumerate(Pubmed_filesList[:]):
    
    # path to each file
    file_path = mypath + file
    
    with open(file_path, "r") as a_file:
        
        # stores annotation line 
        annotation_line2 = []
        
        
        # check if the annotation is what we want (usage, purpose, starts with T )
        for line in a_file:
            
            # grab the type of annotation
            annotataion = line.split()[1:2][0]
            
            if (line.startswith('T') & (annotataion in interest_list)):

                if ( line.split()[2:4] ) not in annotation_line2: 
                    
                    annotation_line2.append(line.split())
                    #print(indx, file, line.split())

                else:
                    pass
        
        fileAn_dict2[file] = annotation_line2     
        
# 3. Merge annotations -----------------------------------------------------------------------


#store all merged annotations with a file name 
# stores MERGED LIST of annotations in each file
merged_dict2 = {}

#stores list of list of all annotations
all_annotations_list2 = []


# for each file in the dict
for key in list(fileAn_dict2):
    
    # stores list of merged annotations
    merged_annotationlist_perfile2 = []
    #print(key)
    """
    compare all annotations to each other and grab those that share the same "start number"
    
    """
    
    lookup_list1 = []
    for ls1 in fileAn_dict2.get(key):
            
            
            lookup_list2 = []
            for ls2 in fileAn_dict2.get(key):
                
                
                # compare each annotation with another but not to itself
                if (ls1 != ls2) :
                    
                    
                    # merge annotations
                    r = mergeList(ls1,ls2)
                    
                    # if the 
                    if len(r) !=0 :
                        #print(ls1[0],ls2[0], ls1[2:4], ls2[2:4])
         
                        if (r[2] not in lookup_list1 ) | (r[3] not in lookup_list1 ):
                            
                            merged_annotationlist_perfile2.append(r) 
                            all_annotations_list2.append(r)
                            
                            lookup_list2.extend(r[2:4])
                            
                        else:
                            pass
            # save already merged list on the lookup list            
            lookup_list1.extend(lookup_list2)  
            #break
                        
                        
    merged_dict2[key] = merged_annotationlist_perfile2

In [15]:
#for file in list(merged_dict2)[:]:
#    print('file name:', file, '|  ',merged_dict2.get(file), '\n\n')

In [16]:
print('size of merged annotatioon list: ', len(all_annotations_list2), '\n')

print('merged annotation list overview:')
all_annotations_list2[:1]

size of merged annotatioon list:  320 

merged annotation list overview:


[['T1', 'Application_Usage:Purpose_Analysis', '15525', '15534', 'SigmaStat']]

## PubMed:  writing merged annotations back to *.ann files

In [17]:
# fileAn_dict -> contains file name -vs- list of  (original) annotations 


print("# of files with ORIGINAL annotations:", len(fileAn_dict2))

print("# of files with MERGED annotations:", len(merged_dict2))

# merged_dict -> contains file name -vs- list of  (merged) annotations 

# of files with ORIGINAL annotations: 100
# of files with MERGED annotations: 100


In [18]:
for file in list(merged_dict2):
    
    path2 = 'SoMeSci/Pubmed_fulltext_Clean/'
    file_path = path2 + file
    
    #print(file_path)
    
    with open(file_path, "w") as f1:
        
        
        for line in merged_dict2.get(file):
            
            txt = '\t'.join(line)
            
            print('Writing To ->',file, '<-',txt)
            
            #write merged annotation to the file 
            #f1.write(txt+ '\n')
            
        #print("-------------------------------------------------")

Writing To -> PMC2790482.ann <- T1	Application_Usage:Purpose_Analysis	15525	15534	SigmaStat
Writing To -> PMC2790482.ann <- T5	Application_Usage:Purpose_DataCollection	23622	23636	Image-Pro Plus
Writing To -> PMC2790482.ann <- T10	Application_Usage:Purpose_DataCollection	23735	23744	Scope-Pro
Writing To -> PMC2790482.ann <- T14	Application_Usage:Purpose_DataPreProcss	24827	24830	IPP
Writing To -> PMC2790482.ann <- T15	Application_Usage:Purpose_Visualization	24913	24922	Photoshop
Writing To -> PMC2790482.ann <- T20	Application_Usage:Purpose_DataPreProcss	25136	25139	IPP
Writing To -> PMC2790482.ann <- T21	Application_Usage:Purpose_DataPreProcss	25842	25854	Office Excel
Writing To -> PMC2790482.ann <- T25	Application_Usage:Purpose_Visualization	25983	25992	SigmaPlot
Writing To -> PMC2790482.ann <- T29	Application_Usage:Purpose_Analysis	26915	26924	Photoshop
Writing To -> PMC2790482.ann <- T30	Application_Usage:Purpose_Analysis	26968	26971	IPP
Writing To -> PMC2790482.ann <- T31	Applicati

### Store the merged annotation lists Into a Data Fram

In [26]:
import pandas as pd

PubMed = pd.DataFrame(columns=['code', 'annotation','start','stop','software'], data=all_annotations_list2)
PubMed

,code,annotation,start,stop,software
0,T1,Application_Usage:Purpose_Analysis,15525,15534,SigmaStat
1,T5,Application_Usage:Purpose_DataCollection,23622,23636,Image-Pro Plus
2,T10,Application_Usage:Purpose_DataCollection,23735,23744,Scope-Pro
3,T14,Application_Usage:Purpose_DataPreProcss,24827,24830,IPP
4,T15,Application_Usage:Purpose_Visualization,24913,24922,Photoshop
...,...,...,...,...,...
315,T1,ProgrammingEnvironment_Usage:Purpose_Analysis,12455,12456,R
316,T3,PlugIn_Usage:Purpose_Analysis,12485,12501,OptimalCutpoints
317,T4,Application_Usage:Purpose_Modelling,11761,11766,Blast
318,T1,Application_Usage:Purpose_DataPreProcss,11418,11429,Trimmomatic


In [88]:
gkk = PubMed.annotation.value_counts()
#gkk

In [87]:
r = PubMed.software.value_counts()
#r

In [89]:
z = PubMed.groupby(['software','annotation']).size().reset_index(name='Count')
z

,software,annotation,Count
0,3dClustsim,PlugIn_Usage:Purpose_Analysis,1
1,3dDeconvolve,PlugIn_Usage:Purpose_Modelling,1
2,3dDetrend,PlugIn_Usage:Purpose_Analysis,1
3,3dFWHMx,PlugIn_Usage:Purpose_Analysis,1
4,3dmaskave,PlugIn_Usage:Purpose_DataCollection,2
...,...,...,...
231,skewer,Application_Usage:Purpose_DataPreProcss,1
232,survival,PlugIn_Usage:Purpose_Analysis,1
233,survminer,PlugIn_Usage:Purpose_Analysis,1
234,tximport,PlugIn_Usage:Purpose_Analysis,1


In [72]:
list_softwareName_Type_Purpose = []

for index, row in z.iterrows():
    
    softwareName = row['software']
    softwareType_Purpose = row['annotation']
    softwareCount = row['Count']
    
    swType = softwareType_Purpose.split(':')[0].split('_')[0]
    swPurp = softwareType_Purpose.split(':')[1].split('_')[1]
    
    lst_ = []
    
    lst_.append(softwareName)
    lst_.append(swType)
    lst_.append(swPurp)
    lst_.append(softwareCount)
    
    list_softwareName_Type_Purpose.append(lst_)
    
    print(f"{softwareName:{30}}|{swType:{25}}|{swPurp:{25}}|{softwareCount:{25}}")

3dClustsim                    |PlugIn                   |Analysis                 |                        1
3dDeconvolve                  |PlugIn                   |Modelling                |                        1
3dDetrend                     |PlugIn                   |Analysis                 |                        1
3dFWHMx                       |PlugIn                   |Analysis                 |                        1
3dmaskave                     |PlugIn                   |DataCollection           |                        2
3dttest+ +                    |PlugIn                   |Analysis                 |                        1
AFNI                          |Application              |Analysis                 |                        2
AFNI                          |Application              |DataCollection           |                        1
AFNI                          |Application              |DataPreProcss            |                        1
AMBERTOOLS         

In [90]:
#list_softwareName_Type_Purpose

In [91]:
import pandas as pd

PubMed_swName_Type_Purpose = pd.DataFrame(columns=['Sw_Name', 'Type','Purpose','count'], data=list_softwareName_Type_Purpose)
PubMed_swName_Type_Purpose

,Sw_Name,Type,Purpose,count
0,3dClustsim,PlugIn,Analysis,1
1,3dDeconvolve,PlugIn,Modelling,1
2,3dDetrend,PlugIn,Analysis,1
3,3dFWHMx,PlugIn,Analysis,1
4,3dmaskave,PlugIn,DataCollection,2
...,...,...,...,...
231,skewer,Application,DataPreProcss,1
232,survival,PlugIn,Analysis,1
233,survminer,PlugIn,Analysis,1
234,tximport,PlugIn,Analysis,1


https://coderedirect.com/questions/323797/how-can-i-merge-rows-by-same-value-in-a-column-in-pandas-with-aggregation-func

https://stackoverflow.com/questions/46636080/merge-rows-within-a-group-together

### Save the df image

In [20]:
import dataframe_image as dfi

PubMed_head = PubMed.head(10)
PubMed_tail = PubMed.tail(10)

# save image of head
PubMed_head = PubMed_head.style.background_gradient()
dfi.export(PubMed_head,"PubMed_head_.png")

# save image of tail
PubMed_tail = PubMed_tail.style.background_gradient()
dfi.export(PubMed_tail,"PubMed_tail_.png")


PubMed.to_csv(r'PubMed_.csv', index = False)

[0110/102642.260474:ERROR:sandbox_linux.cc(376)] InitializeSandbox() called with multiple threads in process gpu-process.
[0110/102642.367323:INFO:headless_shell.cc(653)] Written to file /tmp/tmp6rgdt50_/temp.png.
[0110/102642.493670:ERROR:sandbox_linux.cc(376)] InitializeSandbox() called with multiple threads in process gpu-process.
[0110/102642.605372:INFO:headless_shell.cc(653)] Written to file /tmp/tmps4szyykw/temp.png.


## 3. Merging PLOS and PubMed

In [21]:
#df_PLOS

In [22]:
#PubMed

In [23]:
# merge the two dfs

frames = [df_PLOS,PubMed]
PubMed_PLoS = pd.concat(frames)

# reset index
PubMed_PLoS = PubMed_PLoS.reset_index(drop=True)
#PubMed_PLoS

In [24]:
# EXPORT RESULTS


# save image of tail
PubMed_PLoS_head = PubMed_PLoS.head(10)
PubMed_PLoS_tail = PubMed_PLoS.tail(10)

PubMed_PLoS_head = PubMed_PLoS_head.style.background_gradient()
dfi.export(PubMed_tail,"PubMed_PLoS_head_.png")

PubMed_PLoS_tail = PubMed_PLoS_tail.style.background_gradient()
dfi.export(PubMed_PLoS_tail,"PubMed_PLoS_tail_.png")

# save to csv
PubMed_PLoS.to_csv(r'PubMed_PLoS_.csv', index = False)

[0110/102642.776594:ERROR:sandbox_linux.cc(376)] InitializeSandbox() called with multiple threads in process gpu-process.
[0110/102642.879834:INFO:headless_shell.cc(653)] Written to file /tmp/tmp_lxnoj6z/temp.png.
[0110/102643.002614:ERROR:sandbox_linux.cc(376)] InitializeSandbox() called with multiple threads in process gpu-process.
[0110/102643.119014:INFO:headless_shell.cc(653)] Written to file /tmp/tmp6gtkq1mk/temp.png.


In [93]:
#with open('PMC3366939.ann','r' ) as f:
    #print(f.readlines(), '\n')
    #for x in f.readlines():
        #print(x)

In [94]:
PubMed_PLoS

,code,annotation,start,stop,software
0,T1,Application_Usage:Purpose_Modelling,3670,3681,DS modeling
1,T2,Application_Usage:Purpose_DataCollection,5677,5683,pCLAMP
2,T3,Application_Usage:Purpose_Analysis,1203,1211,ENSEMBLE
3,T1,Application_Usage:Purpose_DataCollection,1447,1450,RIU
4,T2,Application_Usage:Purpose_Analysis,8512,8517,Stata
...,...,...,...,...,...
1677,T1,ProgrammingEnvironment_Usage:Purpose_Analysis,12455,12456,R
1678,T3,PlugIn_Usage:Purpose_Analysis,12485,12501,OptimalCutpoints
1679,T4,Application_Usage:Purpose_Modelling,11761,11766,Blast
1680,T1,Application_Usage:Purpose_DataPreProcss,11418,11429,Trimmomatic


In [92]:
y = PubMed_PLoS.groupby(['software','annotation']).size().reset_index(name='Count')
y

,software,annotation,Count
0,2LD,Application_Usage:Purpose_Analysis,1
1,3dBandpass,PlugIn_Usage:Purpose_DataPreProcss,1
2,3dClustsim,PlugIn_Usage:Purpose_Analysis,1
3,3dDeconvolve,PlugIn_Usage:Purpose_Modelling,1
4,3dDespike,PlugIn_Usage:Purpose_DataPreProcss,1
...,...,...,...
944,tract-based spatial statistics,PlugIn_Usage:Purpose_Analysis,1
945,tximport,PlugIn_Usage:Purpose_Analysis,1
946,vcftools,Application_Usage:Purpose_Analysis,1
947,vegan,PlugIn_Usage:Purpose_Analysis,1


In [96]:
list_softwareName_Type_Purpose2 = []

for index, row in y.iterrows():
    
    softwareName = row['software']
    softwareType_Purpose = row['annotation']
    softwareCount = row['Count']
    
    swType = softwareType_Purpose.split(':')[0].split('_')[0]
    swPurp = softwareType_Purpose.split(':')[1].split('_')[1]
    
    lst_ = []
    
    lst_.append(softwareName)
    lst_.append(swType)
    lst_.append(swPurp)
    lst_.append(softwareCount)
    
    list_softwareName_Type_Purpose2.append(lst_)
    
    print(f"{softwareName:{30}}|{swType:{25}}|{swPurp:{25}}|{softwareCount:{25}}")

2LD                           |Application              |Analysis                 |                        1
3dBandpass                    |PlugIn                   |DataPreProcss            |                        1
3dClustsim                    |PlugIn                   |Analysis                 |                        1
3dDeconvolve                  |PlugIn                   |Modelling                |                        1
3dDespike                     |PlugIn                   |DataPreProcss            |                        1
3dDetrend                     |PlugIn                   |Analysis                 |                        1
3dFDR                         |PlugIn                   |Modelling                |                        1
3dFWHMx                       |PlugIn                   |Analysis                 |                        1
3dmaskave                     |PlugIn                   |DataCollection           |                        2
3dttest+ +         

In [100]:
#list_softwareName_Type_Purpose2

In [99]:
import pandas as pd

PubMedPLoS_swName_Type_Purpose = pd.DataFrame(columns=['Sw_Name', 'Type','Purpose','count'], 
                                              data=list_softwareName_Type_Purpose)

PubMedPLoS_swName_Type_Purpose

,Sw_Name,Type,Purpose,count
0,2LD,Application,Analysis,1
1,3dBandpass,PlugIn,DataPreProcss,1
2,3dClustsim,PlugIn,Analysis,1
3,3dDeconvolve,PlugIn,Modelling,1
4,3dDespike,PlugIn,DataPreProcss,1
...,...,...,...,...
944,tract-based spatial statistics,PlugIn,Analysis,1
945,tximport,PlugIn,Analysis,1
946,vcftools,Application,Analysis,1
947,vegan,PlugIn,Analysis,1
